# 你的第一个实验
### 请阅读本节内容。即使内容较长，这些信息对你的准备工作也非常重要。

## 你的第一个前沿大语言模型项目

让我们在几分钟内构建一个有用的大语言模型解决方案。

在本课程结束时，你将构建一个由7个代理组成的自主智能体AI解决方案，这些代理协同工作以解决业务问题。一切都会循序渐进！我们将从一些较小的项目开始...

我们的目标是编写一个新型的网页浏览器。给它一个URL，它就会返回一个摘要。

在开始之前，你应该已经完成了[PC](../SETUP-PC.md)或[Mac](../SETUP-mac.md)的环境设置，并且希望你已经在项目根目录下启动了jupyter lab，同时激活了你的环境。

## 如果你是Jupyter Lab新手

欢迎来到数据科学实验的精彩世界！一旦你使用过Jupyter Lab，你就会想知道没有它之前是怎么生活的。只需点击每个包含代码的"单元格"，比如这段文字下面的单元格，然后按Shift+Return执行该单元格。根据需要，你可以使用工具栏中的+按钮添加单元格，打印变量值，或尝试不同的变化。

我编写了一个名为[Jupyter指南](Guide%20to%20Jupyter.ipynb)的笔记本，帮助你更熟悉Jupyter Labs，包括添加Markdown注释，使用`!`运行shell命令，以及使用`tqdm`显示进度。




In [ ]:
# 导入所需的库

import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI

# 如果运行这个单元格时出现错误，请查看故障排除笔记本！

# 连接到OpenAI

下一个单元格是我们加载.env文件中的环境变量并连接到OpenAI的地方。

## 如果遇到问题，请进行故障排除：

请查看本文件夹中的[故障排除](troubleshooting.ipynb)笔记本，获取逐步的代码来识别根本原因并修复它！

如果你做了更改，请尝试重启"内核"（这个笔记本背后的python进程），方法是：内核菜单 >> 重启内核并清除所有单元格的输出。然后从顶部开始重新尝试这个笔记本。

对API成本有任何担忧？请参阅README中的说明 - 成本应该很小，而且你可以随时控制它。你也可以使用Ollama作为免费替代方案，我们会在第2天讨论这个。

In [ ]:
# 加载.env文件中的环境变量

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

# 检查密钥

if not api_key:
    print("未找到API密钥 - 请查看本文件夹中的故障排除笔记本以识别和修复问题！")
elif not api_key.startswith("sk-proj-"):
    print("找到了API密钥，但它不是以sk-proj-开头的；请检查你是否使用了正确的密钥 - 参见故障排除笔记本")
elif api_key.strip() != api_key:
    print("找到了API密钥，但看起来在开头或结尾可能有空格或制表符字符 - 请删除它们 - 参见故障排除笔记本")
else:
    print("找到API密钥，看起来目前一切正常！")

In [ ]:
openai = OpenAI()

# 如果这不起作用，请尝试内核菜单 >> 重启内核并清除所有单元格的输出，然后从这个笔记本的顶部开始运行单元格。
# 如果它仍然不起作用（太糟糕了！），请查看本文件夹中的故障排除笔记本获取完整说明

# 让我们快速调用一个前沿模型来开始，作为预览！

In [ ]:
# 给你一个预览 -- 用这些消息调用OpenAI就这么简单。如果有任何问题，请查看故障排除笔记本。

message = "你好，GPT！这是我给你的第一条消息！你好！"
response = openai.chat.completions.create(model="gpt-4o-mini", messages=[{"role":"user", "content":message}])
print(response.choices[0].message.content)

## 好的，让我们继续我们的第一个项目

In [ ]:
# 让我们创建一个函数来获取网页内容

def get_webpage_content(url):
    try:
        # 发送HTTP GET请求
        response = requests.get(url)
        response.raise_for_status()  # 如果请求失败则抛出异常
        
        # 使用BeautifulSoup解析HTML
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # 移除所有脚本和样式元素
        for script in soup(["script", "style"]):
            script.decompose()
            
        # 获取文本
        text = soup.get_text()
        
        # 分割成行并去除前导和尾随空白
        lines = (line.strip() for line in text.splitlines())
        
        # 去除空行
        chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
        text = ' '.join(chunk for chunk in chunks if chunk)
        
        return text
        
    except Exception as e:
        return f"错误：{str(e)}"

# 让我们测试一下这个函数
url = "https://www.iloveaws.cn"
content = get_webpage_content(url)
print(f"网页内容的长度：{len(content)}个字符")
print("
前100个字符：")
print(content[:100])

# 让我们创建一个函数来总结内容

我们将使用OpenAI的GPT-4来总结网页内容。

In [ ]:
def summarize_content(content):
    try:
        # 准备提示
        prompt = f"请用中文总结以下文本的要点：

{content}"
        
        # 调用OpenAI API
        response = openai.chat.completions.create(
            model="gpt-4o-mini",
            messages=[{"role": "user", "content": prompt}]
        )
        
        # 返回总结
        return response.choices[0].message.content
        
    except Exception as e:
        return f"错误：{str(e)}"

# 让我们测试总结函数
summary = summarize_content(content)
print("总结：")
print(summary)

# 让我们把它们组合在一起

现在我们有了获取网页内容和总结内容的函数，让我们创建一个主函数来完成整个过程。

In [ ]:
def summarize_webpage(url):
    print(f"正在处理URL：{url}")
    
    # 获取网页内容
    content = get_webpage_content(url)
    if content.startswith("错误"):
        return content
    
    print(f"成功获取了{len(content)}个字符的内容
")
    
    # 总结内容
    summary = summarize_content(content)
    if summary.startswith("错误"):
        return summary
    
    return summary

# 让我们用一些不同的URL测试它
urls = [
    "https://www.example.com",
    "https://www.wikipedia.org"
]

for url in urls:
    print("
---
")
    summary = summarize_webpage(url)
    print("总结：")
    print(summary)
    print("
---
")

# 恭喜！

你已经创建了一个简单但功能强大的工具，可以：
1. 获取任何网页的内容
2. 使用最先进的AI来总结内容
3. 以易于理解的格式呈现结果

## 下一步

你可以通过以下方式扩展这个项目：
1. 添加错误处理来处理无效的URL
2. 添加对需要认证的网页的支持
3. 添加对不同语言的支持
4. 创建一个简单的Web界面
5. 添加缓存以避免重复处理相同的URL

## 关于API成本的说明

请记住，每次调用OpenAI API都会产生成本。虽然成本很小，但在处理大量URL时要注意这一点。

## 最后的想法

这个简单的项目展示了如何将网络抓取与AI结合起来创建有用的工具。想象一下将这种能力与其他技术结合起来可以构建什么！